In [ ]:
import os
import pandas as pd
import preprocess
from preprocess import *
from indigo import *
import re
import csv
import itertools
import numpy as np
import subprocess
from sklearn.decomposition import PCA
import warnings
    
homedir = os.environ['HOME']
repo_path = '%s/repos/slic_matlab' % homedir
nlbc_path ='%s/ref_data/nlbc_test' % repo_path

def get_info_msms(molecule,probe_radius = 1.4,density=2.0,molXYZR = "test.xyzr",
                  path_to_nlbc_data = nlbc_path,
                  msmsPath = '/Users/Ali/Downloads/msms_MacOSX_2.6.1/msms.MacOSX.2.6.1'):
    """Molecular surface data from msms"""    
    pwd = os.getcwd()
    cp_cmd = 'cp -rf /Users/Ali/repos/setschenow-data/nonpolar/get_surf_info.sh %s' % pwd
    subprocess.Popen(cp_cmd, shell=True).wait()
    chmod_cmd = 'chmod +x ./get_surf_info.sh'
    subprocess.Popen(chmod_cmd, shell=True).wait()
    arr = np.zeros((4,1)).ravel()
    molXYZR_surf = "surf_%s"%(molXYZR)
    msms_cmd = "%s%s%s%s%3.1f%s%3.1f%s%s > msmslog" % (msmsPath,' -if ',molXYZR,' -no_header -de ',density,
                                                 ' -prob ',probe_radius,' -of ',molXYZR_surf)
    subprocess.Popen(msms_cmd, shell=True).wait()
    info_cmd="./get_surf_info.sh msmslog"
    subprocess.Popen(info_cmd, shell=True).wait()
    if os.stat('surf_data').st_size == 0:
        return arr
    arr = np.loadtxt('surf_data', unpack=True).ravel()
    return arr

def get_area_centroid_from_points(points):
    vec1 = points[0] - points[1]
    vec2 = points[0] - points[2]
    centroid = np.sum(points, axis=0)/3
    cross = 0.5*np.cross(vec1,vec2)
    area = np.sqrt(cross@cross)
    return area,centroid,cross/np.linalg.norm(cross)

def get_dist(p1,p2):
    dist_vec = p1-p2
    return np.sqrt(dist_vec@dist_vec)

def get_orbital_info(solute,path=nlbc_path):
    file_name = os.path.join(path,solute,'test.prmtop')
    data=[]
    buff=[]
    with open(file_name, "r") as ifile:
        for line in ifile:
            if line.startswith("%FORMAT(20a4)"):
                data.append(next(ifile).split())
                buff.append(next(ifile).split())
    l=[]
    if buff[-2][0]!="%FLAG":
        l.append(data[-2])
        l.append(buff[-2])
        data_out = list(itertools.chain(*l))
    else:
        data_out=data[-2]
        
    if ('no' in data_out and 'o' in data_out):
        indices = [i for i, x in enumerate(data_out) if x == "o"]
        for index in indices:
            data_out[index]="on"
    return list(np.asarray(data_out))



orbital_df = pd.read_csv('%s/ref_data/atom_type_dict.csv'%repo_path)
atom_type_dict = {}
for i in range(len(orbital_df.index)):
    atom_type_dict[orbital_df.iloc[:,0][i]]=[orbital_df.iloc[:,x][i] for x in range(1,len(orbital_df.columns))]

ref_csv = '%s/ref_data/reference_data_small_molecules_mobley.csv' % repo_path
ref_df = pd.read_csv(ref_csv)
test_solutes_mobley = ref_df.solute.values
n = len(test_solutes_mobley)
data = np.zeros((n,4))
orbital_df = pd.read_csv('%s/ref_data/atom_type_dict.csv'%repo_path)
atom_type_dict = {}
for i in range(len(orbital_df.index)):
    atom_type_dict[orbital_df.iloc[:,0][i]]=[orbital_df.iloc[:,x][i] for x in range(1,len(orbital_df.columns))]
prmtop_df = orbital_df.set_index('prmtop_info')
atom_Bondii_dict = {}
hbond_info = pd.read_csv('%s/ref_data/hbond.csv'%repo_path)
non_acceptor_prefixes = ('b','c','h','i','p','s')
donor_prefixes = ('h')
for k in range(len(test_solutes_mobley)):
    print(test_solutes_mobley[k])
    solute = test_solutes_mobley[k]
    os.chdir(os.path.join(nlbc_path,solute))
    atom_percentile = []
    atom_type_percentile = []
    prmtop_info = get_orbital_info(test_solutes_mobley[k],nlbc_path)
    file_name = open('test_4.srf','r').readlines()[3][2:-1]
    points = np.loadtxt('%s.vert'%file_name,dtype='float',usecols=(0,1,2))
    faces = np.loadtxt('%s.face'%file_name,dtype='int',usecols=(0,1,2))-1
    pqr_list = [x.split() for x in open('test.pqr','r').readlines()]
    pqr = np.array(pqr_list)[:,5:].astype('float')
    charge = get_mol_data_from_pqr()[:,8]
    atom_name_id = get_mol_data_from_pqr()[:,2]
    pos = pqr[:,:3]
    charge = pqr[:,-2]
    radii= pqr[:,-1]
    SES_solute = ref_df['SESA_Mob'][k]
    areas = np.zeros(pqr.shape[0])
    tot_areas = 0
    assigned_area = 0
    all_atom_data = np.asarray([atom_type_dict.get(x) for x in prmtop_info])
    num_tris = faces.shape[0]
    num_atoms = pqrData.shape[0]
    tot_covered = np.zeros((num_tris,num_atoms))
    closest_atom_centers_to_surf_elements = [[""] for x in range(num_tris)]
    dist_to_center = np.zeros((num_tris,num_atoms))
    min_dist_center_to_vert = np.zeros(num_tris)
    unique_atom_types = list(np.unique(all_atom_data[:,0]))
    num_unique_atoms_types = len(unique_atom_types)
    area_s = np.zeros(num_atoms)
    for i in range(num_tris):
        area, centroid, unit_notmal_to_triangle = get_area_centroid_from_points(points[faces[i]])
        tot_areas += area
        for j in range(num_atoms):
            center = np.array([float(x) for x in pqrData[j][5:8]])
            dist_to_center=get_dist(centroid,center)
            c_to_c = centroid-center
            d = c_to_c/np.linalg.norm(c_to_c)
            inner = d@unit_notmal_to_triangle
            if np.isclose(inner,1,rtol=5e-02) and np.isclose(radii[j],dist_to_center,rtol=5e-02):
                tot_covered[i,j] += area
                assigned_area += area
    for jj in range(num_atoms):
        barea = np.round(4*np.pi*float(radii[jj])**2,3)
        area_s[jj] = np.round(sum([tot_covered[x,jj] for x in range(num_tris)]),3)
        atom_percentile.append([atom_name_id[jj],all_atom_data[jj,0],area_s[jj],barea,charge[jj]])
    ll = [[] for x in range(num_unique_atoms_types)]
    factor = tot_areas/assigned_area
    for l in range(num_unique_atoms_types):
        ll[l].append(unique_atom_types[l])
        for hh in range(num_atoms):
            if atom_percentile[hh][1] == unique_atom_types[l]:
                ll[l].append(np.round(atom_percentile[hh][-3]*factor/atom_percentile[hh][-2],3))
        atom_type_percentile.append(ll[l])
    num_donors = hbond_info.iloc[k,1]
    num_acceptors = hbond_info.iloc[k,2]
    q_sorted_ind = np.argsort( charge.ravel() )
    types_sorted = list(np.asarray(prmtop_info)[q_sorted_ind])
    types_sorted_copy = list(np.asarray(prmtop_info)[q_sorted_ind])
    for a_type in types_sorted[:]:
        if a_type.startswith(non_acceptor_prefixes):
            types_sorted.remove(a_type)
    if num_acceptors!=0:
        acceptors = types_sorted[:num_acceptors]
    else:
        acceptors = []
    unique_acceptors = np.unique(acceptors)
    acceptor_data = []
    for acceptor in unique_acceptors:
        acceptor_data.append([acceptor,acceptors.count(acceptor)])
    for a_type in types_sorted_copy[:]:
        if not a_type.startswith(donor_prefixes):
            types_sorted_copy.remove(a_type)
    if num_donors!=0:
        donors = types_sorted_copy[-num_donors:]
    else:
        donors = []
    unique_donors = np.unique(donors)
    donor_data = []
    for donor in unique_donors:
        donor_data.append([donor,donors.count(donor)])
    assigned_area = np.round(assigned_area,3)
    tot_areas = np.round(tot_areas,3)
    atom_Bondii_dict[test_solutes_mobley[k]] = [atom_percentile,atom_type_percentile,\
                                               acceptor_data,donor_data,\
                                               assigned_area,tot_areas]
    os.chdir(nlbc_path)
os.chdir(repo_path)